In [5]:
TEAM_NAME = "alex-team"

# UCHICAGO WORDLE TOURNAMENT

Welcome! This is the starting point for implementing your Wordle bot. Follow along, and reach out to Kathir and Akash with questions :)

In [ ]:
from wordle_tournament_lib import UChicagoWordleBotBase, WordleHint
from collections import Counter

class WordleBot(UChicagoWordleBotBase):
    
    def __init__(self, _team_id):
        """Initialize the bot with a team ID.
        
        Note: Do NOT modify this method's signature - there is some funky Rust stuff 
        that happens behind the scenes. Any object-specific fields should be tracked 
        as we show below with loading the guess corpus (TODO).
        """
        with open('./corpus.txt', 'r') as f:
            self.possible_guesses = f.read().splitlines()
        
        with open('./possible_answers.txt', 'r') as f:
            self.possible_answers = f.read().splitlines()

        self.debug = False
            
    def guess(self, hints: list[WordleHint]) -> str:
        """Return a 5 letter word that you think is the target word given a list of previous 
        guesses/results (see WordleHint type for details).

        Guesses should be valid 5 letter words. Invalid strings will be ignored by our
        evaluation, but will still count towards the number of guesses you make.
        
        Example: 
            # TODO: gotta think of idiomatic way to show an example
        """
        ### Imports & setup
        from collections import defaultdict
        words_remaining = self.possible_answers.copy()

        ### helper functions
        def filter_green(letter, i, words_remaining):
            return [word for word in words_remaining if word[i] == letter]

        def filter_gray(letter, i, words_remaining, letter_states):
            if letter_states[letter] == {'X'}:
                return [w for w in words_remaining if letter not in w]
            else:
                return [w for w in words_remaining if w[i] != letter]
        
        def filter_yellow(letter: str, index: int, words_remaining):
            #TODO watch out for multiple yellows of the same letter
            return [word for word in words_remaining if letter in word and word[index] != letter]

        # def find_best_word(words_remaining):
        #     #naive find best
        #     if words_remaining:
        #         res = words_remaining.pop(0)
        #         if self.debug: print(f"\nGuessing: {res}")
        #         return res 
        #     else: raise ValueError("No words left")

        def find_best_word(words_remaining, used_letters="TODO", known_present="TODO"):
            freq = Counter()
            for w in words_remaining:
                freq.update(set(w))

            def score_word(word):
                score = 0
                for letter in set(word):
                    score += freq[letter]
                    if letter in "aeiouy":
                        score += 0.5
                    if letter not in used_letters:
                        score += 1.0
                    if letter in known_present:
                        score += 1.5

                score -= 1
                return score

            return max(words_remaining, key=score_word)

        ### main guessing logic
        if not hints:
            return "slate"
        for hint in hints:
            if self.debug:
                print("\ncurrent hint:")
                hint.visualize_hint()

            letter_states = defaultdict(set)
            for i, (letter, h) in enumerate(hint.word_hint_pairs):
                letter_states[letter].add(h)

            for i, (letter, h) in enumerate(hint.word_hint_pairs):
                if self.debug: copyremaining = words_remaining.copy()

                if h == 'O':
                    words_remaining = filter_green(letter, i, words_remaining)
                elif h == 'X':
                    words_remaining = filter_gray(letter, i, words_remaining, letter_states)
                else:
                    words_remaining = filter_yellow(letter, i, words_remaining)

                if self.debug: diff = list(set(copyremaining) - set(words_remaining))
                if self.debug: print(f"symb: {h} || remains: {words_remaining[:5]} || removed:{diff[:5]}")
        return find_best_word(words_remaining)
        

## WordleHint

Your bot uses hint objects, which contain info about the word and hints for a given previous guess. In the code block below, we have some code that accesses a WordleHint that might help out in building, testing, and debugging your solution.

In [24]:
from wordle_tournament_lib import WordleHint

h = WordleHint("hello", "OO~XX")
print(f"h.word: {h.word}")
print(f"h.hints: {h.hints}")
print(f"h.word_hint_pairs: {h.word_hint_pairs}")

h.visualize_hint()

h.word: hello
h.hints: OO~XX
h.word_hint_pairs: [('h', 'O'), ('e', 'O'), ('l', '~'), ('l', 'X'), ('o', 'X')]
H E L L O
🟩 🟩 🟨 ⬜ ⬜


## Testing
The block below will test your bot's strategy on an example word and return how many guesses it took (capped to avoid infinite loop).

In [25]:
test_bot = WordleBot(TEAM_NAME)

test_bot.evaluate_on_word(answer="windy", logging=True)

Evaluating bot on answer: windy
----------------------------------------------------
S L A T E
⬜ ⬜ ⬜ ⬜ ⬜
I R O N Y
🟨 ⬜ ⬜ 🟨 🟩
D I N G Y
🟨 🟩 🟩 ⬜ 🟩
W I N D Y
🟩 🟩 🟩 🟩 🟩


4

Now, we will try to run your bot on the entire corpus locally - we are returned the avg number of guesses it took your bot. Note that this is different from the server-side score, which factors in word frequencies as a weight.

In [26]:
local_submission_bot = WordleBot(TEAM_NAME)
local_submission_bot.evaluate(grade_local=True)

Beginning evaluation (local grading)
Team alex-team local eval completed.
Average number of guesses (unweighted) = 3.63


3.6332613390928725

## Submission
Running the block below will submit the bot to our server for evaluation. Score is calculated as the average number of guesses it takes the bot to correctly guess the word (capped at 20).

In [57]:
submission_bot = WordleBot(TEAM_NAME)

# This will take some time to run, and will hit our servers + put you on the leaderboard.
# Re-running will replace your old submission if it's better, and do nothing otherwise.
# DO NOT RUN THIS unless you're ready to submit to the leaderboard.
submission_bot.evaluate(grade_local=False)

Beginning evaluation (remote grading)
Sending mock start signal to server for team alex-team


RuntimeError: Failed to send request to server: error sending request for url (http://localhost:8080/api/guesses)